In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
spark_context = SparkContext("local", "first_app")
spark = SparkSession(spark_context)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sql_context = SQLContext(spark_context)

In [2]:
text_file = sql_context.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', quote='"', delimiter=',').load("/home/jovyan/work/TraficoAereoGt.csv")
rdd_filter = text_file.rdd.map(tuple)
rdd_landings = rdd_filter.map(lambda word: (word[0].split("/")[1], word[5]))
rdd_landings.take(5)

[('01', 195), ('02', 112), ('03', 159), ('04', 193), ('05', 82)]

In [3]:
rdd_landings = rdd_landings.reduceByKey(lambda a,b: a+b)
print(rdd_landings.collect())

[('01', 6034), ('02', 5912), ('03', 5600), ('04', 5723), ('05', 5690), ('06', 5476), ('07', 5972), ('08', 5850), ('09', 5425), ('10', 5347), ('11', 6467), ('12', 5862)]


In [4]:
rdd_landings = spark_context.parallelize(rdd_landings.sortBy(lambda a: a[1], False).take(3))
print(rdd_landings.collect())

[('11', 6467), ('01', 6034), ('07', 5972)]


In [5]:
landing_month = rdd_landings.map(lambda x: x[0])
landing_quantity = rdd_landings.map(lambda x: x[1])
print(landing_month.collect())
print(landing_quantity.collect())

['11', '01', '07']
[6467, 6034, 5972]


In [6]:
fig = go.Figure(data=go.Pie(labels=landing_month.collect(),values=landing_quantity.collect()))
fig.update_layout(title_text='Top 3 months with the highest outbound passengers',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('report4.html', auto_open=True)